In [7]:
import pandas as pd 
import os


In [5]:
def load_all_json_gz_to_dataframe(directory): #TODO: do it in spark 
    all_data = pd.DataFrame()
    for filename in os.listdir(directory):
        if filename.endswith(".json.gz"):
            file_path = os.path.join(directory, filename)
            df = pd.read_json(file_path, compression='gzip', lines=True)
            filtered_df = df[(df['selftext'] != '[removed]') & 
                            (df['selftext'] != '[deleted]') &
                            (df['num_comments'].astype(int) > 10) & 
                            ((df['link_flair_text'] == 'Asshole') | (df['link_flair_text'] == 'Not the A-hole')) & 
                            (df['num_comments'] > 10)
                            ]
            # alot of the posts have their data removed
            all_data = pd.concat([all_data, filtered_df], ignore_index=True)

    return all_data
directory_path = 'reddit-subset/submissions'
df = load_all_json_gz_to_dataframe(directory_path)

In [6]:
# Separate the groups
asshole_group = df[df['link_flair_text'] == 'Asshole']
not_asshole_group = df[df['link_flair_text'] == 'Not the A-hole']

# Size of the smaller group
asshole_size = len(asshole_group)

# Randomly sample from the larger group to match the size of the smaller group
sampled_not_asshole = not_asshole_group.sample(n=asshole_size)

# Combine the samples with the smaller group
balanced_sample = pd.concat([asshole_group, sampled_not_asshole])


# balanced_sample['selftext'] = balanced_sample['selftext'].apply(lambda x: x.encode().decode('unicode-escape'))
balanced_sample.to_json('output/filtered_and_balanced_submissions.json')
